In [2]:
import pandas as pd
import yfinance as yf

sp500 = yf.download("^GSPC", start="2021-04-26", end="2023-04-25", interval="1h")

ModuleNotFoundError: No module named 'pandas'

In [ ]:
sp500 = sp500['Adj Close'] - sp500['Open']